### Install PySpark

In [5]:
!apt-get install openjdk-8-jdk-headless -qq


In [6]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


### Get Kaggle Dataset

In [8]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os

os.environ["KAGGLE_USERNAME"] = "hajerasiddiqui"
os.environ["KAGGLE_KEY"] = "1c827f0cbe0a1b6e8c2413fdcdb6073f"

In [10]:
!kaggle datasets download -d ealaxi/paysim1

 99% 176M/178M [00:05<00:00, 38.8MB/s]
100% 178M/178M [00:05<00:00, 35.4MB/s]


In [11]:
!unzip paysim1.zip

Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [30]:
data = pd.read_csv("PS_20174392719_1491204439457_log.csv")
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [26]:
# Load the dataset file into a DataFrame
df = spark.read.csv("PS_20174392719_1491204439457_log.csv", header=True) #, inferSchema=True)

# Display the DataFrame
df.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

In [27]:
data_subset = df.limit(100)
print(data_subset.toPandas())


   step      type     amount     nameOrig oldbalanceOrg newbalanceOrig  \
0     1   PAYMENT    9839.64  C1231006815      170136.0      160296.36   
1     1   PAYMENT    1864.28  C1666544295       21249.0       19384.72   
2     1  TRANSFER      181.0  C1305486145         181.0            0.0   
3     1  CASH_OUT      181.0   C840083671         181.0            0.0   
4     1   PAYMENT   11668.14  C2048537720       41554.0       29885.86   
..  ...       ...        ...          ...           ...            ...   
95    1  TRANSFER  710544.77   C835773569           0.0            0.0   
96    1  TRANSFER  581294.26   C843299092           0.0            0.0   
97    1  TRANSFER   11996.58   C605982374           0.0            0.0   
98    1   PAYMENT     2875.1  C1412322831       15443.0        12567.9   
99    1   PAYMENT    8586.98  C1305004711        3763.0            0.0   

       nameDest oldbalanceDest newbalanceDest isFraud isFlaggedFraud  
0   M1979787155            0.0          

In [15]:
# import pandas as pd
# num_rows = df.shape[0]
# num_cols = df.shape[1]
# print(num_rows)
# print(num_cols)



num_rows = df.count()
num_cols = len(df.columns)

print("Number of rows:", num_rows)
print("Number of columns:", num_cols)


Number of rows: 6362621
Number of columns: 11


[link text](https://)

In [29]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
diabetes.target

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

In [36]:
from pyspark.sql import SparkSession
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, mean_squared_error
from pyspark.sql.functions import broadcast
import pandas as pd
import numpy as np


# Create a SparkSession
spark = SparkSession.builder \
    .appName("BroadcastAndRunModels") \
    .getOrCreate()

# Load the diabetes dataset from scikit-learn
diabetes = load_diabetes()

# # Convert the features and target to a Pandas DataFrame
# df_features = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
# df_target = pd.DataFrame(data=diabetes.target, columns=['target'])
# df_train = pd.concat([df_features, df_target], axis=1)

# converted_data = data_subset.toPandas()

# df_features = pd.DataFrame(data=converted_data.to_numpy(), columns=data_subset.feature_names)
# df_target = pd.DataFrame(data=data_subset.target, columns=['target'])
# df_train = pd.concat([df_features, df_target], axis=1)

df_train = pd.read_csv("PS_20174392719_1491204439457_log.csv").loc[1:100]
df_train = df_train.rename(columns={'isFraud': 'target'})
print(df_train)

# Broadcast the training dataset
broadcast_train = spark.sparkContext.broadcast(df_train)

# Define the models to run
models = [
    LogisticRegression(),
    Ridge(),
    Lasso()
]

# Define a function to run a model on a partition
def run_model_on_partition(rows):
    # Get the broadcasted training dataset
    train_data = broadcast_train.value

    # Convert the partition rows to a Pandas DataFrame
    #df_partition = pd.DataFrame(rows, columns=df_features.columns)

    # Split the data into features and target
    X = train_data.drop('target', axis=1)
    y = train_data['target']

    # Train and evaluate each model on the partition
    results = []
    for model in models:
        model.fit(X, y)

        if isinstance(model, LogisticRegression):
            y_pred = model.predict(X)
            accuracy = accuracy_score(y, y_pred)
            results.append(('Logistic Regression', accuracy))
        else:
            y_pred = model.predict(X)
            mse = mean_squared_error(y, y_pred)
            results.append((type(model).__name__, mse))

    return results

# Create an RDD from the training dataset
rdd_train = spark.sparkContext.parallelize(df_train.values.tolist())

# Apply the model function on each partition of the RDD
results_rdd = rdd_train.mapPartitions(run_model_on_partition)

# Collect the results from all partitions
results = results_rdd.collect()

# Print the results
for model_name, result in results:
    print(f"Model: {model_name}, Result: {result}")




     step      type     amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
1       1   PAYMENT    1864.28  C1666544295        21249.0        19384.72   
2       1  TRANSFER     181.00  C1305486145          181.0            0.00   
3       1  CASH_OUT     181.00   C840083671          181.0            0.00   
4       1   PAYMENT   11668.14  C2048537720        41554.0        29885.86   
5       1   PAYMENT    7817.71    C90045638        53860.0        46042.29   
..    ...       ...        ...          ...            ...             ...   
96      1  TRANSFER  581294.26   C843299092            0.0            0.00   
97      1  TRANSFER   11996.58   C605982374            0.0            0.00   
98      1   PAYMENT    2875.10  C1412322831        15443.0        12567.90   
99      1   PAYMENT    8586.98  C1305004711         3763.0            0.00   
100     1   PAYMENT     871.75  C1003206025        19869.0        18997.25   

        nameDest  oldbalanceDest  newbalanceDest  target  isFla

Py4JJavaError: ignored

## Pushing code to repo

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change working directory to the desired location
%cd /content/drive/MyDrive/ColabNotebooks/CS6240
#/content/drive/My_Drive/Colab_Notebooks/CS_6240/Project_Work

# Clone the GitHub repository
!git clone https://github.com/saicharan-thirandas/cs6240-course-project.git

# Change working directory to the cloned repository
%cd cs6240-course-project


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/CS6240'
/content/drive/MyDrive/cs6240-course-project/cs6240-course-project
Cloning into 'cs6240-course-project'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 6), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (32/32), 5.78 KiB | 14.00 KiB/s, done.
/content/drive/MyDrive/cs6240-course-project/cs6240-course-project/cs6240-course-project


In [ ]:
# Replace the filename with name of the file to be updated
!git add Project_Work
# Add a commit message
!git commit -m "test push"
# Push the changes to the remote repository
!git push origin master

fatal: failed to stat '/content/drive/MyDrive/cs6240-course-project/cs6240-course-project/cs6240-course-project': No such file or directory
fatal: failed to stat '/content/drive/MyDrive/cs6240-course-project/cs6240-course-project/cs6240-course-project': No such file or directory
fatal: failed to stat '/content/drive/MyDrive/cs6240-course-project/cs6240-course-project/cs6240-course-project': No such file or directory
